# Fractionally Differentiated Features

### Loading Libraries

In [1]:
# Randomness
import random

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
from pandas import Timestamp

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.io as pio
%matplotlib inline

# Date & Time
from datetime import datetime, timedelta

# Typing
from typing import Tuple, List, Dict, Union, Optional, Any, Generator

# Scikit-Learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve

# Scientific Statistical Python
from scipy.stats import jarque_bera

## The Method

### Iterative Estimation
#### `Weighting Function`

In [3]:
def get_weights(d: float, size: int) -> np.ndarray:
    '''
    Computing the weights for differentiating the series
    
        Parameters:
            d (float): differentiating factor
            size (int): length of weights array
            
        Returns:
            w (np.ndarray): array contatining weights
    '''
    w = [1.0]
    for k in range(1, size):
        w_ = -w[-1] / k * (d - k + 1)
        w.append(w_)
    w = np.array(w[::-1]).reshape(-1, 1)
    return w


def plot_weights(dRange: list, nPlots: int, size: int) -> None:
    w = pd.DataFrame()
    for d in np.linspace(dRange[0], dRange[1], nPlots):
        w_ = get_weights(d, size)
        w_ = pd.DataFrame(w_, index=range(w_.shape[0])[::-1], columns=[d])
        w = w.join(w_, how='outer')
    fig, ax = plt.subplots(figsize=(11, 7))
    ax.plot(w)
    ax.set_xlabel('$k$')
    ax.set_ylabel('$w_k$')
    ax.legend(np.round(np.linspace(dRange[0], dRange[1], nPlots), 2), loc='lower right')
    plt.show()

## Implementation

### Expanding Window
#### `Standard Fracdiff (Expanding Window)`

In [5]:
def frac_diff(series: pd.DataFrame, d: float, thres: float = 0.01) -> pd.DataFrame:
    w = get_weights(d, series.shape[0])
    w_ = np.cumsum(abs(w))
    w_ /= w_[-1]
    skip = w_[w_ > thres].shape[0]
    
    df = {}
    for name in series.columns:
        seriesF, df_ = series[[name]].fillna(method='ffill').dropna(), \
                       pd.Series(index=np.arange(series.shape[0]), dtype=object)
        for iloc in range(skip, seriesF.shape[0]):
            loc = seriesF.index[iloc]
            if not np.isfinite(series.loc[loc, name]):
                continue    # exclude NAs
            df_[loc] = np.dot(w[-(iloc + 1):, :].T, seriesF.loc[:loc])[0, 0]
        df[name] = df_.dropna().copy(deep=True)
    df = pd.concat(df, axis=1)
    return df

### Fixed-Width Window Francdiff
#### `New Fixed-Width Francdiff Method`

In [6]:
def get_weights_ffd(d: float, thres: float) -> np.ndarray:
    w, k = [1.0], 1
    while True:
        w_ = -w[-1] / k * (d - k + 1)
        if abs(w_) < thres:
            break
        w.append(w_)
        k += 1
    w = np.array(w[::-1]).reshape(-1, 1)
    return w

In [7]:
## Stationarity with Maximun Memory Preservation
#### `Finding The Minimum D Value that Passes The ADF Test`